In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import PIL
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')
import tensorflow_addons as tfa
from tensorflow.keras.metrics import top_k_categorical_accuracy, categorical_accuracy

In [2]:
root = "D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Complete Dataset"

In [3]:
train_dir = root+'/Hair Removed Images/train'
val_dir = root+'/Hair Removed Images/val'
test_dir = root+'/Hair Removed Images/test'
datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

# test_datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

val_data = datagen.flow_from_directory(val_dir,target_size=(224,224),batch_size=8,class_mode='categorical',seed=42)

test_data = datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=8,class_mode='categorical',shuffle=False)

Found 40253 images belonging to 7 classes.
Found 1000 images belonging to 7 classes.
Found 1006 images belonging to 7 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model=MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(7,activation='softmax')(x) #final layer with softmax activation

In [5]:
model=Model(inputs=base_model.input,outputs=preds)

In [6]:
def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

lrReduction=ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.5,
                              patience=2,
                              verbose=1,
                              mode='auto',
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, verbose=1)

In [7]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',
                                                                        top_2_acc,
                                                                        top_3_acc,
                                                                        tf.metrics.AUC(curve='ROC'),
                                                                        tfa.metrics.MatthewsCorrelationCoefficient(num_classes=7)])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, None, 3 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [8]:
step_size_train=train_data.n//train_data.batch_size
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=step_size_train,
                   validation_data=val_data,
                    epochs=50, 
                    verbose = 1,
                    callbacks=[lrReduction,early_stopping])

Epoch 1/50
5031/5031 [==============================] - 578s 113ms/step - loss: 1.3326 - accuracy: 0.4838 - top_2_acc: 0.6822 - top_3_acc: 0.8142 - auc: 0.8455 - MatthewsCorrelationCoefficient: 0.4004 - val_loss: 2.5272 - val_accuracy: 0.4550 - val_top_2_acc: 0.7290 - val_top_3_acc: 0.8630 - val_auc: 0.7736 - val_MatthewsCorrelationCoefficient: 0.1847
Epoch 2/50
5031/5031 [==============================] - 430s 85ms/step - loss: 0.9713 - accuracy: 0.6279 - top_2_acc: 0.8177 - top_3_acc: 0.9159 - auc: 0.9209 - MatthewsCorrelationCoefficient: 0.5669 - val_loss: 1.6874 - val_accuracy: 0.3990 - val_top_2_acc: 0.7090 - val_top_3_acc: 0.8300 - val_auc: 0.8025 - val_MatthewsCorrelationCoefficient: 0.2577
Epoch 3/50
5031/5031 [==============================] - 443s 88ms/step - loss: 0.8144 - accuracy: 0.6892 - top_2_acc: 0.8617 - top_3_acc: 0.9425 - auc: 0.9442 - MatthewsCorrelationCoefficient: 0.6380 - val_loss: 1.1864 - val_accuracy: 0.5320 - val_top_2_acc: 0.7970 - val_top_3_acc: 0.9170 - v

5031/5031 [==============================] - 397s 79ms/step - loss: 0.0663 - accuracy: 0.9765 - top_2_acc: 0.9972 - top_3_acc: 0.9997 - auc: 0.9993 - MatthewsCorrelationCoefficient: 0.9726 - val_loss: 0.7724 - val_accuracy: 0.8440 - val_top_2_acc: 0.9390 - val_top_3_acc: 0.9850 - val_auc: 0.9601 - val_MatthewsCorrelationCoefficient: 0.6840
Epoch 24/50
5031/5031 [==============================] - 396s 79ms/step - loss: 0.0594 - accuracy: 0.9783 - top_2_acc: 0.9975 - top_3_acc: 0.9999 - auc: 0.9994 - MatthewsCorrelationCoefficient: 0.9747 - val_loss: 0.8074 - val_accuracy: 0.8290 - val_top_2_acc: 0.9370 - val_top_3_acc: 0.9830 - val_auc: 0.9617 - val_MatthewsCorrelationCoefficient: 0.6536
Epoch 25/50
5031/5031 [==============================] - 395s 79ms/step - loss: 0.0567 - accuracy: 0.9796 - top_2_acc: 0.9978 - top_3_acc: 0.9999 - auc: 0.9994 - MatthewsCorrelationCoefficient: 0.9763 - val_loss: 0.8399 - val_accuracy: 0.8320 - val_top_2_acc: 0.9450 - val_top_3_acc: 0.9860 - val_auc: 0.

In [9]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(train_data)

5032/5032 [==============================] - 106s 21ms/step - loss: 0.0209 - accuracy: 0.9935 - top_2_acc: 0.9995 - top_3_acc: 1.0000 - auc: 0.9999 - MatthewsCorrelationCoefficient: 0.9924


In [10]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(val_data)

125/125 [==============================] - 3s 22ms/step - loss: 0.8884 - accuracy: 0.8340 - top_2_acc: 0.9390 - top_3_acc: 0.9840 - auc: 0.9567 - MatthewsCorrelationCoefficient: 0.6645


In [11]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(test_data)

126/126 [==============================] - 12s 99ms/step - loss: 0.8788 - accuracy: 0.8101 - top_2_acc: 0.9414 - top_3_acc: 0.9821 - auc: 0.9579 - MatthewsCorrelationCoefficient: 0.6232 4s - loss: 1.1683 - accuracy: 0.7436 - top_2_acc: 0.9215 - top_3_acc: 0.9824 - auc


In [12]:
from sklearn import metrics

true_test_labels=test_data.labels

true_test_labels=np.array(true_test_labels)

np.unique(true_test_labels,return_counts=True)

pred_test_labels=model.predict(test_data)

pred_test_labels=np.argmax(pred_test_labels,axis=1)

metrics.accuracy_score(true_test_labels,pred_test_labels)

report = metrics.classification_report(true_test_labels,pred_test_labels)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.74      0.77        34
           1       0.66      0.77      0.71        52
           2       0.66      0.60      0.63       111
           3       0.86      0.50      0.63        12
           4       0.60      0.38      0.47       112
           5       0.86      0.93      0.90       670
           6       0.92      0.73      0.81        15

    accuracy                           0.81      1006
   macro avg       0.77      0.67      0.70      1006
weighted avg       0.80      0.81      0.80      1006



In [13]:
# model.save("D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Saved Models\mobileNet_v2_new.h5")